In [2]:
import os
import imageio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from scipy import stats, signal
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
import seaborn as sns
import cv2

## Lectura de los datasets

In [54]:
def abrir_imagenes(path):
    imagen_ant_em=np.flipud(imageio.imread(path + 'ANT_EM.dcm'))
    imagen_ant_sc=np.flipud(imageio.imread(path + 'ANT_SC.dcm'))
    imagen_pos_em=np.flipud(imageio.imread(path + 'POST_EM.dcm'))
    imagen_pos_sc=np.flipud(imageio.imread(path + 'POST_SC.dcm'))
    meta=imagen_ant_em.meta
    imagen=np.stack((imagen_ant_em,imagen_ant_sc,imagen_pos_em,imagen_pos_sc))
    return imagen, meta

In [55]:
#path_dicom_1='./DISTINTOSTIEMPOS/1h/'
#path_dicom_15='./DISTINTOSTIEMPOS/15h/'
#path_dicom_40='./DISTINTOSTIEMPOS/40h/'
#path_dicom_64='./DISTINTOSTIEMPOS/64h/'
path_dicom_1='Data set/distintos_tiempos/1h/'
path_dicom_15='Data set/distintos_tiempos/15h/'
path_dicom_40='Data set/distintos_tiempos/40h/'
path_dicom_64='Data set/distintos_tiempos/64h/'

In [56]:
imagen_1, meta_1=abrir_imagenes(path_dicom_1)
imagen_15, meta_15=abrir_imagenes(path_dicom_15)
imagen_40, meta_40=abrir_imagenes(path_dicom_40)
imagen_64, meta_64=abrir_imagenes(path_dicom_64)
pos=['ANT_EM','ANT_SC','POST_EM','POST_SC']

## 1. Defina la elección del dataset y describa el mismo a partir de un análisis exploratorio que considere pertinente

### Se selecciona un dataset compuesto por imágenes de cámara gama tomadas a 4 tiempos (1h, 15h, 40h, 64h) para 2 posiciones (anterior y posterior) y EM y SC

Toda la información de las imagenes se encuentran almacenada en en la variable meta_x (x=1,15,40,64). Esta variable almacena los metadatos de la imágen original DICOM.

In [57]:
meta_1

Dict([('TransferSyntaxUID', '1.2.840.10008.1.2.1'),
      ('SOPClassUID', '1.2.840.10008.5.1.4.1.1.20'),
      ('SOPInstanceUID', '1.2.840.113619.2.170.1.2.0.13072009150430562.18713'),
      ('StudyDate', '20090713'),
      ('SeriesDate', '20090713'),
      ('AcquisitionDate', '20090713'),
      ('ContentDate', '20090713'),
      ('StudyTime', '103711.00 '),
      ('SeriesTime', '170440.00 '),
      ('AcquisitionTime', '170440.00 '),
      ('ContentTime', '170440.00 '),
      ('Modality', 'NM'),
      ('Manufacturer', 'GE MEDICAL SYSTEMS'),
      ('InstitutionName', ''),
      ('StudyDescription', 'User&Lu177&Dosimetria 177Lu'),
      ('SeriesDescription', 'WB 1 ore'),
      ('PatientName', 'TI'),
      ('PatientID', 'TELu1772800190DOSIM'),
      ('PatientBirthDate', ''),
      ('PatientSex', 'F '),
      ('PatientSize', 0.0),
      ('PatientWeight', 0.0),
      ('StudyInstanceUID',
       '1.2.840.113619.2.170.1.2.0.13072009081643828.4265'),
      ('SeriesInstanceUID',
       '1.2.840

In [58]:
print ('El estudio se realizo el' ,meta_1['StudyDate'][6:8],'del',meta_1['StudyDate'][4:6],'del',meta_1['StudyDate'][0:4])
print ('A las',meta_1['SeriesTime'][0:2],':',meta_1['SeriesTime'][2:4],':',meta_1['SeriesTime'][4:6])

El estudio se realizo el 13 del 07 del 2009
A las 17 : 04 : 40


In [59]:
print ('El estudio se realizo el' ,meta_15['SeriesDate'][6:8],'del',meta_15['SeriesDate'][4:6],'del',meta_15['SeriesDate'][0:4])
print ('A las',meta_15['SeriesTime'][0:2],':',meta_15['SeriesTime'][2:4],':',meta_15['SeriesTime'][4:6])

El estudio se realizo el 14 del 07 del 2009
A las 09 : 07 : 12


In [60]:
print ('El estudio se realizo el' ,meta_40['SeriesDate'][6:8],'del',meta_40['SeriesDate'][4:6],'del',meta_40['SeriesDate'][0:4])
print ('A las',meta_40['SeriesTime'][0:2],':',meta_40['SeriesTime'][2:4],':',meta_40['SeriesTime'][4:6])

El estudio se realizo el 15 del 07 del 2009
A las 08 : 50 : 06


In [61]:
print ('El estudio se realizo el' ,meta_64['SeriesDate'][6:8],'del',meta_64['SeriesDate'][4:6],'del',meta_64['SeriesDate'][0:4])
print ('A las',meta_64['SeriesTime'][0:2],':',meta_64['SeriesTime'][2:4],':',meta_64['SeriesTime'][4:6])

El estudio se realizo el 16 del 07 del 2009
A las 08 : 04 : 21


In [62]:
print ('La imágen tiene', meta_1['Columns'], 'pixels de ancho y',meta_1['Rows'],'pixeles de largo')

La imágen tiene 256 pixels de ancho y 1024 pixeles de largo


In [63]:
print ('El tamaño de los pixeles es:',meta_1['PixelSpacing'][0],'x',meta_1['PixelSpacing'][0],'mm2')

El tamaño de los pixeles es: 2.20903 x 2.20903 mm2


## 2. Visualice los datos e identifique posibles clústers por diferentes métodos de clustering


In [1]:
import pickle

In [3]:
pickle.load("data.pkl")

TypeError: file must have 'read' and 'readline' attributes

In [4]:
pkl_file = open('data.pkl', 'rb')

In [5]:
data1 = pickle.load(pkl_file)

In [6]:
data1

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)